<a href="https://colab.research.google.com/github/luknda/q-lora-bert-imdb/blob/master/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install torch
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [2]:
import torch
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training, AutoPeftModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
# 1. Load a pre-trained model and evaluate it
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # print(f"Real labels: {labels.tolist()}")  # Used for testing
    # print(f"Predicted labels: {preds.tolist()}")  # Used for testing
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def load_and_evaluate_model(model_name, dataset_name):

    dataset = load_dataset(dataset_name)

    # Loading model and tokenizer
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
    tokenizer = BertTokenizerFast.from_pretrained(model_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Sample dataset, too long otherwise and Im running low on compute units sadly
    train_dataset = tokenized_datasets["train"].shuffle(seed=41).select(range(10000)) # Not really used here
    eval_dataset = tokenized_datasets["test"].shuffle(seed=41).select(range(2000))

    # Trainer stuff, training args dont really matter here
    training_args = TrainingArguments(
        output_dir="./results_b",
        learning_rate=2e-5,
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs',
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    results = trainer.evaluate()

    return model, tokenizer, results


In [4]:
# 2. Perform lightweight fine-tuning using a pre-trained model
def fine_tune_with_lora(model, tokenizer, dataset_name, save_directory):

    dataset = load_dataset(dataset_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    train_dataset = tokenized_datasets["train"].shuffle(seed=41).select(range(10000))
    eval_dataset = tokenized_datasets["test"].shuffle(seed=41).select(range(2000))

    # LoRA config
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
    )

    peft_model = get_peft_model(model, peft_config)

    peft_model.print_trainable_parameters()

    # Trainer stuff
    training_args = TrainingArguments(
        output_dir="./results_f",
        learning_rate=2e-5,
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs_f',
        logging_steps=10,
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    # print(trainer.state.log_history)
    # results = trainer.evaluate()

    # Save stuff
    try:
        peft_model.save_pretrained(save_directory)
        print("Model and tokenizer saved successfully.")

    except Exception as e:
        print(f"Error saving model and tokenizer: {e}")

    return peft_model

In [5]:
# 3. Perform lightweight fine-tuning (with quantization) using a pre-trained model
def fine_tune_with_lora_q(model_name, tokenizer, dataset_name, save_directory):

    dataset = load_dataset(dataset_name)

    # Q
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16  # input type is torch.float16, default seems to be torch.float32
    )

    model_q = BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        quantization_config=quantization_config
    )

    # Prepare quantized model for peft training
    model_q = prepare_model_for_kbit_training(model_q)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    train_dataset = tokenized_datasets["train"].shuffle(seed=41).select(range(10000))
    eval_dataset = tokenized_datasets["test"].shuffle(seed=41).select(range(2000))

    # LoRA config
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
    )

    peft_model = get_peft_model(model_q, peft_config)

    peft_model.print_trainable_parameters()

    # Trainer stuff
    training_args = TrainingArguments(
        output_dir="./results_fq",
        learning_rate=2e-5,
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs_fq',
        logging_steps=10,
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    # print(trainer.state.log_history)
    # results = trainer.evaluate()

    # Save stuff
    try:
        peft_model.save_pretrained(save_directory)
        print("Model and tokenizer saved successfully.")

    except Exception as e:
        print(f"Error saving model and tokenizer: {e}")

    return peft_model, model_q

In [6]:
def inference_lora(tokenizer, save_directory, dataset_name):

    dataset = load_dataset(dataset_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    train_dataset = tokenized_datasets["train"].shuffle(seed=41).select(range(10000))
    eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10000))

    # base_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

    peft_model = AutoPeftModelForSequenceClassification.from_pretrained(
        save_directory,
        num_labels=2
    )

    # Trainer stuff
    training_args = TrainingArguments(
        output_dir="./results_inf",
        learning_rate=2e-5,
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs_inf',
        logging_steps=10,
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    results = trainer.evaluate()
    print("Inference done")

    return results

In [7]:
def inference_qlora(tokenizer, save_directory, dataset_name):

    dataset = load_dataset(dataset_name)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    train_dataset = tokenized_datasets["train"].shuffle(seed=41).select(range(10000))
    eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10000))

    # Q
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    # Load
    peft_model = AutoPeftModelForSequenceClassification.from_pretrained(
        save_directory,
        quantization_config=quantization_config,
        num_labels=2
    )

    # Trainer stuff
    training_args = TrainingArguments(
        output_dir="./results_inf_q",
        learning_rate=2e-5,
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs_inf_q',
        logging_steps=10,
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    results = trainer.evaluate()
    print("Inference done")

    return results

In [8]:
# Main
if __name__ == "__main__":
    model_name = "bert-base-uncased"
    dataset_name = "imdb"
    save_directory = "./my_fine_tuned_lora_model"
    save_directory_q = "./my_fine_tuned_lora_model_q"

    # 1. Load and evaluate bert on imdb dataset for sequence classification.
    original_model, tokenizer, original_results = load_and_evaluate_model(model_name, dataset_name)
    print("Original model performance:", original_results)

    # 2. Fine-tune bert with LoRA on imdb dataset for sequence classification, also run some evaluations
    fine_tuned_model = fine_tune_with_lora(original_model, tokenizer, dataset_name, save_directory)
    #print("Fine-tuned model performance:", fine_tuned_results)

    # 3. Fine-tune bert with QLoRA on imdb dataset for sequence classification, also run some evaluations
    fine_tuned_model_q, model_q = fine_tune_with_lora_q(model_name, tokenizer, dataset_name, save_directory_q)
    #print("Fine-tuned (with quantization) model performance:", fine_tuned_results_q)

    # 4. Load the models, proceed with inference
    results_lora = inference_lora(tokenizer, save_directory, dataset_name)
    print(f"LoRA results: {results_lora}\n")
    results_qlora = inference_qlora(tokenizer, save_directory_q, dataset_name)
    print(f"QLoRA results: {results_qlora}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Inference done
QLoRA results: {'eval_loss': 0.2415771484375, 'eval_accuracy': 0.9075, 'eval_f1': 0.9097120546608102, 'eval_precision': 0.8877881501238332, 'eval_recall': 0.932746196957566, 'eval_runtime': 19.7413, 'eval_samples_per_second': 506.553, 'eval_steps_per_second': 7.953}
